# Red Neuronal en PyTorch para California Housing


Este notebook implementa una red neuronal usando PyTorch para resolver un problema de regresión sobre el dataset de California Housing.
Incluye:
- Preprocesamiento avanzado con `preprocessing.py`
- Escalado de la variable objetivo
- Arquitectura profunda
- Early stopping manual
- Evaluación sobre test


In [2]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from preprocessing import get_preprocessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)


Usando dispositivo: cpu


In [3]:

housing = pd.read_csv("housing.csv")
X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"].copy()

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)


In [4]:

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).astype(np.float32)
y_valid_scaled = y_scaler.transform(y_valid.values.reshape(-1, 1)).astype(np.float32)
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).astype(np.float32)


In [5]:

preprocessor = get_preprocessor(housing)
X_train_prep = preprocessor.fit_transform(X_train).astype(np.float32)
X_valid_prep = preprocessor.transform(X_valid).astype(np.float32)
X_test_prep = preprocessor.transform(X_test).astype(np.float32)

X_train_tensor = torch.tensor(X_train_prep).to(device)
y_train_tensor = torch.tensor(y_train_scaled).to(device)
X_valid_tensor = torch.tensor(X_valid_prep).to(device)
y_valid_tensor = torch.tensor(y_valid_scaled).to(device)
X_test_tensor = torch.tensor(X_test_prep).to(device)
y_test_tensor = torch.tensor(y_test_scaled).to(device)


In [6]:

class HousingNet(nn.Module):
    def __init__(self, input_dim):
        super(HousingNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

model = HousingNet(X_train_prep.shape[1]).to(device)


In [7]:

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
best_rmse = float('inf')
patience = 10
trigger = 0

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_valid_tensor)
        val_loss = criterion(val_outputs, y_valid_tensor)
        val_rmse = np.sqrt(val_loss.item())

    print(f"Epoch {epoch+1}, Val RMSE: {val_rmse:.2f}")
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_model_state = model.state_dict()
        trigger = 0
    else:
        trigger += 1
        if trigger >= patience:
            print("Early stopping.")
            break


Epoch 1, Val RMSE: 0.98
Epoch 2, Val RMSE: 0.94
Epoch 3, Val RMSE: 0.90
Epoch 4, Val RMSE: 0.84
Epoch 5, Val RMSE: 0.79
Epoch 6, Val RMSE: 0.73
Epoch 7, Val RMSE: 0.69
Epoch 8, Val RMSE: 0.67
Epoch 9, Val RMSE: 0.65
Epoch 10, Val RMSE: 0.64
Epoch 11, Val RMSE: 0.62
Epoch 12, Val RMSE: 0.61
Epoch 13, Val RMSE: 0.59
Epoch 14, Val RMSE: 0.59
Epoch 15, Val RMSE: 0.58
Epoch 16, Val RMSE: 0.57
Epoch 17, Val RMSE: 0.57
Epoch 18, Val RMSE: 0.58
Epoch 19, Val RMSE: 0.56
Epoch 20, Val RMSE: 0.55
Epoch 21, Val RMSE: 0.56
Epoch 22, Val RMSE: 0.56
Epoch 23, Val RMSE: 0.56
Epoch 24, Val RMSE: 0.57
Epoch 25, Val RMSE: 0.57
Epoch 26, Val RMSE: 0.56
Epoch 27, Val RMSE: 0.55
Epoch 28, Val RMSE: 0.55
Epoch 29, Val RMSE: 0.54
Epoch 30, Val RMSE: 0.54
Epoch 31, Val RMSE: 0.53
Epoch 32, Val RMSE: 0.53
Epoch 33, Val RMSE: 0.53
Epoch 34, Val RMSE: 0.53
Epoch 35, Val RMSE: 0.53
Epoch 36, Val RMSE: 0.54
Epoch 37, Val RMSE: 0.54
Epoch 38, Val RMSE: 0.54
Epoch 39, Val RMSE: 0.53
Epoch 40, Val RMSE: 0.52
Epoch 41,

In [8]:

model.load_state_dict(best_model_state)
model.eval()
with torch.no_grad():
    y_test_pred_scaled = model(X_test_tensor).cpu().numpy()
    y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print("✅ RMSE test:", rmse_test)

# Guardar predicciones
pd.DataFrame({
    "y_real": y_test.values,
    "y_pred": y_test_pred.flatten()
}).to_csv("predicciones_test_pytorch.csv", index=False)


✅ RMSE test: 55694.82824356644
